# Токсичные комментарии

Целью проекта является получение моделей для выявления токсичных описаний и комментариев к продуктам интернет-магазина. Для обучения моделей имеется массив описаний и комментариев с пометкой токсичны они или нет.

Для достижения цели, будут выполнены следующие шаги:

**<a href="#prepare">1. Подготовка.</a>** Данные будут загружены, проверены и подготовлены к машинному обучению.

**<a href="#learning">2. Обучение.</a>** Будет обучено несколько видов моделей и измерено качество их предсказаний.

**<a href="#conclusion">3. Выводы.</a>** Будут сделаны выводы о возможности и перспективах выявления токсичных текстов.

*Примечания:*

*1. Подробнее шаги получения моделей отражены в автоматическом оглавлении тетради.*

*2. Выполнение тетради может занять **длительное время** (от 10 до 20 мин. в зависимости от производительности компьютера).*

<a id="prepare"></a>

## Подготовка данных

Подключим все необходимые для дальнейшей работы модули:

In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.metrics import auc
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier 
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

rnd_state=12345

### Загрузка и проверка

Загрузим данные из файла:

In [2]:
data = pd.read_csv('data/toxic_comments.csv') # на локальной машине
    
#data = data.sample(data.shape[0]//30) # для быстрой проверки всей тетради

Проверим структуру и первые несколько записей:

In [3]:
data.head(3)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


Структура и значения соответствуют ожидаемым.

Проверим размер загруженного:

In [4]:
data.shape

(159571, 2)

Объём данных соответствует задаче машинного обучения.

Проверим наличие пропусков в данных:

In [5]:
data.isnull().sum()

text     0
toxic    0
dtype: int64

Пропущенных значений нет:

Проверим типы значений:

In [6]:
data.dtypes

text     object
toxic     int64
dtype: object

Типы столбцов соответствуют хранимой в них информации.

Проверим дублирование данных:

In [7]:
data.duplicated().sum(), data['text'].duplicated().sum()

(0, 0)

Дублей нет.

Проверим целевой признак на наличие некорректных значений:

In [8]:
data['toxic'].unique()

array([0, 1])

Целевой признак принимает значение 0 либо 1. Некорректных значений нет.

**Итог:** исходные данные загружены, проверены и готовы к использованию.

### Лемматизация

Попробуем выяснить язык комментариев:

In [9]:
data.sample(10)

,text,toxic
141442,Nobody needs to explain anything. The 300 albu...,0
19082,""" August 2010 (UTC)\nI suppose one can assume ...",0
6418,Russian language==\n\nSee Wikipedia:Reference ...,0
51538,Joseph Stalin\nPlease stop adding unsourced no...,0
92625,"""\n\nYou really do need to justify that, BoxOf...",0
94685,"(UTC)\n\n I recognise what you are saying, but...",0
79126,"That would be fair, provided he advertises it,...",0
94861,", 23 May 2006 (UTC)\n\n Those borders are well...",0
16245,William Bedell is a good example of the attitu...,0
100817,Sld For Scrap \n\nhttp://today.seattletimes.co...,0


Случайная выборка говорит о том, что комментарии сделаны на английском языке. Убедимся, что это не случайность и комментариев на русском нет:

In [10]:
data['text'].str.contains(r'[А-Яа-яЁё]').sum() / data.shape[0], \
data['text'].str.contains(r'[А-Яа-яЁё]{2,}\s+[А-Яа-яЁё]{2,}').sum() / data.shape[0]

(0.0016231019420822079, 0.0004010753833716653)

Кириллические символы содержат только 0.16% комментариев, при этом пару русских слов, разделённых пробелом, содержат и того меньше - 0.04% комментариев. Значит, комментарии не русскоязычные, а англоязычные - для лемматизации воспользуемся библиотекой nltk.

Лемматизируем комментарии:

In [11]:
def lemmatize(text):
    try:
        return ' '.join([stemmer.stem(w) for w in word_tokenize(
            re.sub('[^a-zA-Z]', ' ', text))])
    except:
        pass

stemmer = PorterStemmer()
nltk.download('punkt')
data['lemm_text'] = data['text'].apply(lambda text: lemmatize(text))

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Проверим, какое число комментариев не удалось лемматизировать:

In [12]:
data['lemm_text'].isna().sum()

1

Посмотрим текст проблеммного комментария:

In [13]:
if (data['lemm_text'].isna().sum() > 0):
    excepted_idx = data[data['lemm_text'].isna()].index
    excepted_text = data.loc[excepted_idx, 'text'].values[0]
    excepted_text[:300]

Исправим и лемматизируем проблеммный комментарий:

In [14]:
if (data['lemm_text'].isna().sum() > 0):
    corrected_text = excepted_text[:excepted_text.index('yyy')]
    data.loc[excepted_idx, ['text','lemm_text']] = [corrected_text,
        lemmatize(corrected_text)]
    data.loc[excepted_idx,]

**Итог:** все тексты лемматизированы.

### Деление на выборки для МО

Выделим из исходных данных целевые признаки. Перекодируем тексты в векторы, содержащие TF-IDF значения. Разделим данные на обучающую, валидационную и тестовую выборки:

In [15]:
target = data['toxic']
#texts  = data['lemm_text'].values.astype('U')
texts  = data['lemm_text'].values

del data['text']

texts_train, texts_x, target_train, target_x = train_test_split(texts, target, 
    train_size=0.6, random_state=rnd_state)
texts_valid, texts_test, target_valid, target_test = train_test_split(texts_x, target_x,
    train_size=0.5, random_state=rnd_state)

nltk.download('stopwords')
#stop_words = set(stopwords.words('russian')) 
#tf_idf_vectorizer = TfidfVectorizer(stop_words=stop_words) 
tf_idf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), lowercase=True)#, min_df=0.0001)

features_train = tf_idf_vectorizer.fit_transform(texts_train)
features_valid = tf_idf_vectorizer.transform(texts_valid)
features_test  = tf_idf_vectorizer.transform(texts_test)

del texts_x, target_x

[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Итог:** данные для обучения моделей готовы.

<a id="learning"></a>

## Обучение моделей

Перед обучением моделей выберем метрику для оценки их качества. В качестве основной метрики оценки моделей возьмём F1. Она хорошо отражает баланс между точностью и полнотой. В качестве дополнительной (вспомогательной) метрики возьмём площадь под кривой precision-recall. Она позволит оценить уровень превосходства моделей над моделью, которая делает случайные предсказания (для неё метрика равна 0,5).

Обе метрики совсем или мало чувстсвительны к дисбалансу классов, а он в нашем случае имеет место быть:

In [16]:
target.mean()

0.10167887648758234

Определим фукцию вычисления метрик:

In [17]:
def get_f1_n_auc(y_true, y_predicted):
    #recall = recall_score(y_true, y_predicted)
    f1 = f1_score(y_true, y_predicted)
    precisions, recalls, _ = precision_recall_curve(y_true, y_predicted)
    pr_auc = auc(recalls, precisions)
    #return recall, pr_auc
    return f1, pr_auc

Обучим модели вида:

    - линейная регрессия,
    - решающее дерево,
    - случайный лес: scikit-learn, LightGBM, CatBoost, XGBoost.
    
Для моделей каждого вида  создадим по одной подпрограмме. Подпрограммы будут обучать и выбирать модель с лучшими значениями гиперпараметров.

In [18]:
def get_best_logistic_regression_model(x_train, y_train, x_valid, y_valid):
    best_model, best_solver, best_c, best_f1, best_auc = None, 0, 0, 0, 0
    #for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
    for solver in ['newton-cg']: # сужено для ускорения выполнения тетради
        #for c100 in range(40, 101, 5):
        for c100 in range(50, 101, 10): # сужено для ускорения выполнения тетради
            c = c100 / 100
            model = LogisticRegression(random_state=rnd_state, solver=solver,
                    class_weight='balanced', max_iter=1000, C=c)
            model.fit(x_train, y_train)
            y_pred = model.predict(x_valid)
            f1, pr_auc = get_f1_n_auc(y_valid, y_pred)
            if best_f1 < f1:
                best_model, best_solver, best_c, best_f1, best_auc = model, solver, c, f1, pr_auc
        print(solver, end=' ')
    best_solver = 'solver=\'{}\', C={}'.format(best_solver, best_c)
    return best_model, best_solver, best_f1, best_auc

Создадим функцию обучения моделей решающего дерева с подбором максимальной глубины:

In [19]:
def get_best_decision_tree_model(x_train, y_train, x_valid, y_valid):
    best_model, best_depth, best_f1, best_auc = None, 0, 0, 0
    #for depth in range(100, 1001, 100):
    for depth in range(1000, 1001, 100): # диапазон сужен для ускорения выполнения тетради
        model = DecisionTreeClassifier(random_state=rnd_state, max_depth=depth, 
            class_weight='balanced')
        model.fit(x_train, y_train)
        y_pred = model.predict(x_valid)
        f1, pr_auc = get_f1_n_auc(y_valid, y_pred)
        if best_f1 < f1:
            best_model, best_depth, best_f1, best_auc = model, depth, f1, pr_auc
        print(depth, end=' ')
    best_depth = 'depth=\'{}\''.format(depth)
    return best_model, best_depth, best_f1, best_auc

Создадим функцию обучения моделей случайного леса с подбором числа деревьев:

In [20]:
def get_best_random_forest_model(x_train, y_train, x_valid, y_valid):
    best_model, best_estimators, best_f1, best_auc = None, 0, 0, 0
    #for estimators in range(100,1001,100):
    for estimators in range(100,101,100): # сужено для ускорения выполнения тетради
        model = RandomForestClassifier(random_state=rnd_state, n_estimators=estimators,
            class_weight='balanced')
        model.fit(x_train, y_train)
        y_pred = model.predict(x_valid)
        f1, pr_auc = get_f1_n_auc(y_valid, y_pred)
        if best_f1 < f1:
            best_model, best_estimators, best_f1, best_auc = model, estimators, f1, pr_auc
        print(estimators, end=' ')
    best_estimators = 'estimators=\'{}\''.format(best_estimators)
    return best_model, best_estimators, best_f1, best_auc

Создадим функцию обучения моделей случайного леса из библиотеки Light GBM с подбором числа деревьев:

In [21]:
def get_best_lightgbm_model(x_train, y_train, x_valid, y_valid):
    best_model, best_estimators, best_f1, best_auc = None, 0, 0, 0
    #for estimators in range(100,2001,100):
    for estimators in range(1075,1076,25): # сужено для ускорения выполнения тетради
        model = LGBMClassifier(n_estimators=estimators, random_state=rnd_state,
            class_weight='balanced')
        model.fit(x_train, y_train)
        y_pred = model.predict(x_valid)
        f1, pr_auc = get_f1_n_auc(y_valid, y_pred)
        if best_f1 < f1:
            best_model, best_estimators, best_f1, best_auc = model, estimators, f1, pr_auc
        print(estimators, end=' ')
    best_estimators = 'estimators=\'{}\''.format(best_estimators)
    return best_model, best_estimators, best_f1, best_auc

Создадим функцию обучения моделей случайного леса из библиотеки CatBoost с подбором числа деревьев:

In [22]:
class1_weight = (target_train == 0).sum() / (target_train == 1).sum()

def get_best_catboost_model(x_train, y_train, x_valid, y_valid):
    best_model, best_estimators, best_f1, best_auc = None, 0, 0, 0
    #for estimators in range(500,2001,250):
    for estimators in range(1250,1251,100): # сужено для ускорения выполнения тетради
        model = CatBoostClassifier(n_estimators=estimators, random_state=rnd_state,
            class_weights=[1,class1_weight], verbose=False)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_valid)
        f1, pr_auc = get_f1_n_auc(y_valid, y_pred)
        if best_f1 < f1:
            best_model, best_estimators, best_f1, best_auc = model, estimators, f1, pr_auc
        print(estimators, end=' ')
    best_estimators = 'estimators=\'{}\''.format(best_estimators)
    return best_model, best_estimators, best_f1, best_auc

Создадим функцию обучения моделей случайного леса из библиотеки CatBoost с подбором числа деревьев:

In [23]:
def get_best_xgboost_model(x_train, y_train, x_valid, y_valid):
    best_model, best_estimators, best_f1, best_auc = None, 0, 0, 0
    #for estimators in range(500,2001,250):
    for estimators in range(1750,1751,100): # сужено для ускорения выполнения тетради
        model = XGBClassifier(n_estimators=estimators, random_state=rnd_state,
            scale_pos_weight=class1_weight, verbosity=0, use_label_encoder=False)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_valid)
        f1, pr_auc = get_f1_n_auc(y_valid, y_pred)
        if best_f1 < f1:
            best_model, best_estimators, best_f1, best_auc = model, estimators, f1, pr_auc
        print(estimators, end=' ')
    best_estimators = 'estimators=\'{}\''.format(best_estimators)
    return best_model, best_estimators, best_f1, best_auc

Создадим отчёт, куда будем записывать результаты обучения моделей:

In [24]:
report = pd.DataFrame(columns=['model_class', 'model_params', 'f1', 'pr-auc',
       'model_object'])

Создадим функцию добавления результатов обучения в отчёт:

In [25]:
def report_add(report, model_name, model_params, recall, pr_auc, model='-'):
    uid_columns = report.columns[:2]
    report.loc[report.shape[0]] = [model_name, model_params, recall, pr_auc, model]
    # Устранение дублей в случае повторного вызова для тех же моделей (при отладке)
    report = report[~report.duplicated(keep='last', subset=uid_columns)]
    report = report.reset_index(drop=True)
    return report

Обучим модели:

*Внимание! Чтобы сократить время обучения в подпрограммах искусственно сужены диапазоны перебора значений гиперпараметров. Они сужены к тем значениям, при которых получены наилучшие варианты моделей. В подпрограммы добавлены соответствующие комментарии.*

In [26]:
print('ML begin:')

trainer_names = ['Logistic Regression', 'Decision Tree', 'Random Forest', 
                 'LightGBM', 'CatBoost', 'XGBoost']
trainers = [get_best_logistic_regression_model, get_best_decision_tree_model,
            get_best_random_forest_model, get_best_lightgbm_model,
            get_best_catboost_model, get_best_xgboost_model]

#trainer_names = ['XGBoost']
#trainers = [get_best_xgboost_model]

for trainer_name, trainer in zip(trainer_names, trainers):
    print('\n' + trainer_name + '...', end=' ')
    model, model_params, recall, pr_auc = trainer(features_train, target_train, 
        features_valid, target_valid)
    report = report_add(report, trainer_name, model_params, recall, pr_auc, model) 

print('\nML end.')
report[report.columns[:-1]]

ML begin:

Logistic Regression... newton-cg 
Decision Tree... 1000 
Random Forest... 100 
LightGBM... 1075 
CatBoost... 1250 
XGBoost... 1750 
ML end.


,model_class,model_params,f1,pr-auc
0,Logistic Regression,"solver='newton-cg', C=1.0",0.755760,0.773450
1,Decision Tree,depth='1000',0.666572,0.684778
2,Random Forest,estimators='100',0.627764,0.734632
3,LightGBM,estimators='1075',0.778230,0.789110
4,CatBoost,estimators='1250',0.761492,0.775909
5,XGBoost,estimators='1750',0.783931,0.795558


На валидационной выборке все модели проходят тест на адекватность. Их precision-recall AUC заметно больше 0.5 (значения модели, делающей случайные предсказания). Лучшее качество у моделей ... , худшее у Random Forest.

Выполним итоговую проверку моделей на тестовой выборке и дополнительно выведем метрику recall:

In [27]:
test_metrics = report.apply(axis=1, func=lambda row: 
    get_f1_n_auc(target_test, row['model_object'].predict(features_test)))
test_data = pd.DataFrame(zip(*test_metrics)).T
test_data.columns = ['f1-test', 'pr-auc-test']
report = report.join(test_data)
report['recall-test'] = report.apply(axis=1, func=lambda row: 
    recall_score(target_test, row['model_object'].predict(features_test)))
columns_to_show = report.columns.to_list()
columns_to_show.remove('model_object')
report[columns_to_show]

,model_class,model_params,f1,pr-auc,f1-test,pr-auc-test,recall-test
0,Logistic Regression,"solver='newton-cg', C=1.0",0.755760,0.773450,0.752397,0.770790,0.855763
1,Decision Tree,depth='1000',0.666572,0.684778,0.655963,0.674601,0.712773
2,Random Forest,estimators='100',0.627764,0.734632,0.618402,0.726012,0.461682
3,LightGBM,estimators='1075',0.778230,0.789110,0.771180,0.782204,0.786916
4,CatBoost,estimators='1250',0.761492,0.775909,0.759626,0.774249,0.835826
5,XGBoost,estimators='1750',0.783931,0.795558,0.782841,0.794166,0.776012


На тестовой выборке все модели показали тот же уровень качества и адекватности, что и на валидационной выборке (значения метрик практически не изменились).

<a id="conclusion"></a>

## Выводы

Обучение разных видов моделей показало, что автоматическое выявление токсичных текстов в описаниях продуктов и комментариях к ним возможно. Лучшие **модели машинного обучения позволяют выявить примерно 80%-86% токсичных текстов** (метрика recall).

Повысить уровень выявления токсичных текстов можно как путём использования более совершенных моделей (например, модели **BERT**, способной учитывать контекст), так и путём использования **ручной обработки**. В последнем случае модель будет выдавать вероятность того, что текст токсичен, в диапазоне от 0 до 100%. Тексты, для которых вероятность выше заданного верхнего порога, будут автоматически относится к токсичными. Тексты, для которых вероятность окажется ниже заданного нижнего порога, будут автоматически относится к нетоксичным. Остальные комментарии будут проверяться операторами вручную. (Сейчас модели автоматически относят комментарий к токсичным, если вероятность больше 0,5, и к нетоксичным в противном случае.)